In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd #read files
from pathlib import Path

In [5]:
df = pd.read_csv('/Users/Tico/Documents/Masters/LaMona/ModelTraining/elements.csv')

In [13]:
df['name'].value_counts()

name
HP Spectre x360 16-f1500na 2-in-1 16" Laptop Intel i7 12th Gen 16GB 512GB SSD        45
HP Spectre x360 14-ef2502na 13.5" Touchscreen Laptop i5 13th Gen 16GB 512GB Black    30
HP Envy x360 15-fe0514sa 15" OLED Touch Laptop Intel i5 13th Gen 8GB RAM 512GB       30
HP Pavilion x360 14-ek1511sa 14" 2 in 1 Touch Laptop Intel i3 13th Gen 8GB 256GB     15
HP Spectre x360 14-ef2503na 13.5" OLED Touch Laptop i7 13th Gen 16GB RAM 1TB         15
                                                                                     ..
JD Bug Original Street MS130B Foldable Scooter - Reflex Blue                          1
JD Bug Original Street MS130B Foldable Scooter - Matt Black                           1
JD Bug Original Street MS130B Foldable Scooter - Sky Blue                             1
JD Bug Original Street MS130B Foldable Scooter - Pastel Pink                          1
Low Triple Cupboard M2                                                                1
Name: count, Length: 744, d

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=22)
